## ONNX graph, single or double floats

The notebook shows discrepencies obtained by using double floats instead of single float in two cases. The second one involves [GaussianProcessRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.gaussian_process.GaussianProcessRegressor.html).

In [1]:
from jyquickhelper import add_notebook_menu
add_notebook_menu()

## Simple case of a linear regression

A linear regression is simply a matrix multiplication followed by an addition: $Y=AX+B$. Let's train one with [scikit-learn](https://scikit-learn.org/stable/).

In [2]:
from sklearn.linear_model import LinearRegression
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
data = load_boston()
X, y = data.data, data.target
X_train, X_test, y_train, y_test = train_test_split(X, y)
clr = LinearRegression()
clr.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [3]:
clr.score(X_test, y_test)

0.6670395137932994

In [4]:
clr.coef_

array([-1.08807982e-01,  4.18783285e-02,  2.95768444e-02,  1.23352147e+00,
       -1.22136785e+01,  4.15983177e+00, -7.14285476e-03, -1.44446027e+00,
        3.21297163e-01, -1.61442487e-02, -8.54076388e-01,  1.06359543e-02,
       -5.61967175e-01])

In [5]:
clr.intercept_

31.095709912741835

Let's predict with *scikit-learn* and *python*.

In [6]:
ypred = clr.predict(X_test)
ypred[:5]

array([15.8763016 , -0.71631653, 34.93376518, 23.32226277, 13.52997418])

In [7]:
py_pred = X_test @ clr.coef_ + clr.intercept_
py_pred[:5]

array([15.8763016 , -0.71631653, 34.93376518, 23.32226277, 13.52997418])

In [8]:
clr.coef_.dtype, clr.intercept_.dtype

(dtype('float64'), dtype('float64'))

## With ONNX

With *ONNX*, we would write this operation as follows... We still need to convert everything into single floats = float32.

In [9]:
%load_ext mlprodict

In [10]:
from skl2onnx.algebra.onnx_ops import OnnxMatMul, OnnxAdd
import numpy

onnx_fct = OnnxAdd(OnnxMatMul('X', clr.coef_.astype(numpy.float32)),
                   numpy.array([clr.intercept_]),
                   output_names=['Y'])
onnx_model32 = onnx_fct.to_onnx({'X': X_test.astype(numpy.float32)},
                              dtype=numpy.float32)

%onnxview onnx_model32

The next line uses a python runtime to compute the prediction.

In [11]:
from mlprodict.onnxrt import OnnxInference
oinf = OnnxInference(onnx_model32)
ort_pred = oinf.run({'X': X_test})['Y']
ort_pred[:5]

array([15.87630339, -0.71631559, 34.9337676 , 23.32226466, 13.52997575])

And here is the same with [onnxruntime](https://github.com/microsoft/onnxruntime)...

In [12]:
oinf = OnnxInference(onnx_model32, runtime="onnxruntime1")
ort_pred = oinf.run({'X': X_test.astype(numpy.float32)})['Y']
ort_pred[:5]

array([15.876303 , -0.7163162, 34.933765 , 23.322264 , 13.529976 ],
      dtype=float32)

## With double instead of single float

[ONNX](https://onnx.ai/) was originally designed for deep learning which usually uses floats but it does not mean cannot be used. Every number is converted into double floats.

In [13]:
onnx_fct = OnnxAdd(OnnxMatMul('X', clr.coef_.astype(numpy.float64)),
                   numpy.array([clr.intercept_]),
                   output_names=['Y'])
onnx_model64 = onnx_fct.to_onnx({'X': X_test.astype(numpy.float64)},
                              dtype=numpy.float64)

And now the *python* runtime...

In [14]:
oinf = OnnxInference(onnx_model64)
ort_pred = oinf.run({'X': X_test})['Y']
ort_pred[:5]

array([15.8763016 , -0.71631653, 34.93376518, 23.32226277, 13.52997418])

And the *onnxruntime* version of it, not fully supportive of double yet...

In [15]:
try:
    oinf = OnnxInference(onnx_model64, runtime="onnxruntime1")
    ort_pred = oinf.run({'X': X_test.astype(numpy.float64)})['Y']
    ort_pred[:5]
except RuntimeError as e:
    print(e)

[ONNXRuntimeError] : 9 : NOT_IMPLEMENTED : Could not find an implementation for the node Add:Add(7)


## And now the GaussianProcess

This shows a case

In [16]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct
gau = GaussianProcessRegressor(alpha=10, kernel=DotProduct())
gau.fit(X_train, y_train)

GaussianProcessRegressor(alpha=10, copy_X_train=True,
                         kernel=DotProduct(sigma_0=1), n_restarts_optimizer=0,
                         normalize_y=False, optimizer='fmin_l_bfgs_b',
                         random_state=None)

In [17]:
from skl2onnx import to_onnx
onnxgau32 = to_onnx(gau, X_train.astype(numpy.float32), dtype=numpy.float32)
oinf32 = OnnxInference(onnxgau32, runtime="python")
ort_pred32 = oinf32.run({'X': X_test.astype(numpy.float32)})['GPmean']
numpy.squeeze(ort_pred32)[:25]

array([16.4375, -2.    , 33.75  , 23.    , 12.625 , 21.5   ,  3.    ,
       25.375 , 17.    , 21.4375, 30.    , 27.8125, 33.0625, 22.8125,
       16.375 , 19.3125, 19.75  , 24.25  , 22.25  , 25.5   , 34.625 ,
       32.25  ,  9.0625, 17.125 , 42.1875], dtype=float32)

In [18]:
onnxgau64 = to_onnx(gau, X_train.astype(numpy.float64), dtype=numpy.float64)
oinf64 = OnnxInference(onnxgau64, runtime="python")
ort_pred64 = oinf64.run({'X': X_test.astype(numpy.float64)})['GPmean']
numpy.squeeze(ort_pred64)[:25]

array([16.04081028, -2.01646275, 34.10963296, 23.26133518, 13.44183912,
       21.37161805,  2.49549085, 25.18234533, 17.8935267 , 20.88278054,
       30.00257955, 27.97794299, 33.49701693, 22.92631105, 16.6742227 ,
       19.24649946, 20.32735247, 24.13743567, 22.35845436, 25.10696109,
       34.4250675 , 32.68237862,  9.30724677, 17.33682715, 42.38769632])

The differences between the predictions for single floats and double floats...

In [19]:
numpy.sort(numpy.sort(numpy.squeeze(ort_pred32 - ort_pred64)))[-5:]

array([0.54388054, 0.54659484, 0.55471946, 0.63846466, 1.20325194])

Who's right or wrong... The differences between the predictions with the original model...

In [20]:
pred = gau.predict(X_test.astype(numpy.float64))

In [21]:
numpy.sort(numpy.sort(numpy.squeeze(ort_pred32 - pred)))[-5:]

array([0.54388054, 0.54659484, 0.55471946, 0.63846466, 1.20325194])

In [22]:
numpy.sort(numpy.sort(numpy.squeeze(ort_pred64 - pred)))[-5:]

array([0., 0., 0., 0., 0.])

Double predictions clearly wins.

In [23]:
%onnxview onnxgau64

## Saves...

Let's keep track of it.

In [24]:
with open("gpr_dot_product_boston_32.onnx", "wb") as f:
    f.write(onnxgau32.SerializePartialToString())

In [25]:
with open("gpr_dot_product_boston_64.onnx", "wb") as f:
    f.write(onnxgau64.SerializePartialToString())